In [115]:
import os
import cv2
import numpy as np
import pandas as pd
import gc

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from torchvision.models import resnet18, ResNet18_Weights, GoogLeNet, GoogLeNet_Weights
from keras import layers

In [116]:
class CustomDataset(Dataset):
    def __init__(self, root, filenames, mode="train"):
        self.image_root = root
        self.filenames = filenames
        self.mode = mode 
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        self.labels = []
        #要改的地方
        if self.mode == mode:
            #df = pd.read_csv("E:\garypython\drive-download-20230604T142257Z-001\label.csv")
            for i in os.listdir(self.image_root):
                if 'imgN' in i:
                    label_map = {"{}".format(i): "0" }
                    self.labels.append(label_map)
                else:
                    label_map = {"{}".format(i): "1"}
                    self.labels.append(label_map)

    def __len__(self):
        return len(self.filenames)


    def __getitem__(self, index):
        filename = self.filenames[index]
        image_path = os.path.join(self.image_root, filename)
        image = cv2.imread(image_path)
        image = self.transform(image)
        
        label = self.labels[index]
        return image, label
        
        '''
        if self.mode == mode:
            label = self.labels[index]
            return image, label
        else:
            return image'''

In [117]:
workspace = "E:\garypython\drive-download-20230604T142257Z-001"
train_root_old = os.path.join(workspace, "train")

filenames_old = [filename for filename in os.listdir(train_root_old)]

a = 1
b = 1
for i in os.listdir(train_root_old):
    pt = os.path.join(train_root_old , i)
    if 'Non_Autistic' in i:
        img = cv2.imread(pt, cv2.IMREAD_GRAYSCALE)
        img = cv2.Canny(img, 100, 150)
        cv2.imwrite("E:\garypython\drive-download-20230604T142257Z-001/new_train/imgN.{}.jpg".format(a), img)
        a += 1
    else:
        img = cv2.imread(pt, cv2.IMREAD_GRAYSCALE)
        img = cv2.Canny(img, 100, 150)
        cv2.imwrite("E:\garypython\drive-download-20230604T142257Z-001/new_train/imgA.{}.jpg".format(b), img)
        b += 1
    
    
del filenames_old
gc.collect()

train_root_new = os.path.join(workspace, "new_train")

filenames_new = [filename for filename in os.listdir(train_root_new)]

train_filenames = filenames_new[:int(len(filenames_new)*0.8)]
valid_filenames = filenames_new[int(len(filenames_new)*0.8):]

train_dataset = CustomDataset(train_root_new, train_filenames, mode="new_train")
valid_dataset = CustomDataset(train_root_new, valid_filenames, mode="new_train")

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

In [118]:
class ClassificationModel(nn.Module):
    def __init__(self):
        super(ClassificationModel, self).__init__()
        self.vit = models.vision_transformer.vit_b_16(pretrained=True)

    def forward(self, x):
        out = self.vit(x)
        
        return out

In [119]:
device = 'cuda'
num_epoch = 30

model = ClassificationModel()
model = model.to(device)

loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5, weight_decay=0.01)

best_accuracy = 0.0
for epoch in range(num_epoch):
    train_loss = 0.0
    correct, total = 0, 0
    model.train()
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images) 

        batch_loss = loss(outputs, labels)
        batch_loss.backward() #讓我CUDA不會爆掉
        optimizer.step() #改權重

        train_loss += batch_loss.item()
        correct += sum(torch.argmax(outputs, dim=1) == labels).item()#判讀結果，正確就加1
        total += images.shape[0]
    train_accuracy = correct / total * 100

    valid_loss = 0.0
    correct, total = 0, 0
    model.eval()
    for images, labels in valid_loader:
        images = images.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            outputs = model(images)
          
        batch_loss = loss(outputs, labels)
        valid_loss += batch_loss.item()
        correct += sum(torch.argmax(outputs, dim=1) == labels).item()
        total += images.shape[0]
    valid_accuracy = correct / total * 100

    if valid_accuracy > best_accuracy:
        best_accuracy = valid_accuracy
        torch.save(model.state_dict(), "best_weight.pt")

    print(f"Epoch [{epoch+1}/{num_epoch}] - Train Loss: {train_loss/len(train_loader):.4f}, Train Accuracy: {train_accuracy:.2f}%, Valid Loss: {valid_loss/len(valid_loader):.4f}, Valid Accuracy: {valid_accuracy:.2f}%")

KeyError: 'imgN.1192.jpg'

In [ ]:
import cv2
import numpy as np
img = np.zeros((500,500,3), dtype='uint8')   # 快速產生 500x500，每個項目為 [0,0,0] 的三維陣列
img[150:350, 150:350] = [0,0,255]  # 將中間 200x200 的每個項目內容，改為 [0,0,255]
cv2.imwrite('oxxostudio.{}.jpg'.format(1), img)       # 存成 jpg
cv2.imshow('oxxostudio', img)            # 顯示圖片
cv2.waitKey(0)                           # 按下任意鍵停止
cv2.destroyAllWindows()

In [ ]:
kkk = "E:\garypython\drive-download-20230604T142257Z-001\hhh"
a = 1
b = 1
for i in os.listdir(kkk):
    if 'Non_Autistic' in i:
        img = cv2.imread(i, cv2.IMREAD_GRAYSCALE)
        #img = cv2.Canny(img, 100, 150)
        cv2.imshow('gg', img)            # 顯示圖片
        cv2.waitKey(0)                           # 按下任意鍵停止
             
        #cv2.imwrite("imgN.{}.jpg".format(a), img)
        a += 1
    else:
        img = cv2.imread(i, cv2.IMREAD_GRAYSCALE)
        #img = cv2.Canny(img, 100, 150)
        cv2.imshow('gg', img)            # 顯示圖片
        cv2.waitKey(0)                           # 按下任意鍵停止
        #cv2.imwrite("imgA.{}.jpg".format(b), img)
        b += 1
cv2.destroyAllWindows()

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


In [ ]:
i = "E:\garypython\drive-download-20230604T142257Z-001/hhh/Autistic.0.jpg"
img = cv2.imread(i, cv2.IMREAD_GRAYSCALE)
#cv2.imshow('gg', img)
cv2.imwrite("E:\garypython\drive-download-20230604T142257Z-001/hhh/imgA.1.jpg", img)

True